In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import re

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv
/kaggle/input/offensive-language/olid-training-v1.0.csv
/kaggle/input/offensive-language/olid-training-v1.csv


In [2]:
data = pd.read_csv('../input/hate-speech-and-offensive-language-dataset/labeled_data.csv')
data1 = pd.read_csv('../input/offensive-language/olid-training-v1.0.csv',error_bad_lines=False)

b'Skipping line 42: expected 1 fields, saw 2\nSkipping line 113: expected 1 fields, saw 5\nSkipping line 135: expected 1 fields, saw 3\nSkipping line 148: expected 1 fields, saw 3\nSkipping line 176: expected 1 fields, saw 2\nSkipping line 182: expected 1 fields, saw 2\nSkipping line 193: expected 1 fields, saw 3\nSkipping line 229: expected 1 fields, saw 2\nSkipping line 235: expected 1 fields, saw 3\nSkipping line 273: expected 1 fields, saw 2\nSkipping line 298: expected 1 fields, saw 3\nSkipping line 319: expected 1 fields, saw 2\nSkipping line 323: expected 1 fields, saw 2\nSkipping line 349: expected 1 fields, saw 3\nSkipping line 423: expected 1 fields, saw 3\nSkipping line 430: expected 1 fields, saw 2\nSkipping line 449: expected 1 fields, saw 3\nSkipping line 488: expected 1 fields, saw 3\nSkipping line 523: expected 1 fields, saw 3\nSkipping line 539: expected 1 fields, saw 4\nSkipping line 544: expected 1 fields, saw 2\nSkipping line 578: expected 1 fields, saw 3\nSkipping 

In [3]:
data1[['id','tweet','subtask_a','subtask_b','subtask_c']] = data1['id\ttweet\tsubtask_a\tsubtask_b\tsubtask_c'].str.split("\t",expand=True,)
data1.drop(['id\ttweet\tsubtask_a\tsubtask_b\tsubtask_c','id','subtask_b','subtask_c'],axis=1,inplace=True)
data1['target'] = data1['subtask_a']
data1.drop(['subtask_a'],axis=1,inplace=True)
data1['target'].replace(['NOT','OFF'],[1,0],inplace=True)
data['target'] = data['class']
data.drop(['class','offensive_language','hate_speech','neither','Unnamed: 0','count'],axis=1,inplace=True)
data.drop(data[data['target']==0].index,inplace=True)
data['target'].replace([1,2],[0,1],inplace=True)

In [4]:
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    username_regex = ('@[a-zA-Z0-9_]{0,15}')
    rt_regex = ('!* RT|!*RT')
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URL', parsed_text)
    parsed_text = re.sub(username_regex, '@USER', parsed_text)
    parsed_text = re.sub(rt_regex,'',parsed_text)
    return parsed_text


def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()



In [5]:
data.tweet = data.tweet.apply(preprocess)

In [6]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

tweets = data.tweet.apply(tokenizer)

In [7]:
dataset = pd.concat([data,data1])

In [8]:
dataset.index = range(len(dataset))

In [9]:
dataset

,tweet,target
0,@USER: As a woman you shouldn't complain abou...,1
1,@USER: boy dats cold...tyga dwn bad for cuffi...,0
2,@USER Dawg @USER: You ever fuck a bitch and s...,0
3,@USER: @USER she look like a tranny,0
4,@USER: The shit you hear about me might be tr...,0
...,...,...
35993,@USER Sometimes I get strong vibes from people...,0
35994,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,1
35995,@USER And why report this garbage. We don't g...,0
35996,@USER Pussy,0


In [10]:
dataset.to_csv('hate_dataset.csv',index=False)

In [11]:
dataset1 = pd.read_csv('hate_dataset.csv')

In [15]:
dataset1.target.value_counts()

0    23393
1    12605
Name: target, dtype: int64